# STEWARDSHIP-BASED SUSCEPTIBILITY REPORTS

Run all cells at once or indiviudally. You will be prompted for input twice (the jovyan password to connect to mysql and to select an individual suscpetibility report). 

# PULL MICROBIOLOGY DATA FROM MIMIC-II

Enter the password (you know it) for jovyan when prompted. 

In [ ]:
import pymysql
import pandas as pd
import numpy as np
import getpass

conn = pymysql.connect(host="mysql",
                       port=3306,
                       user="jovyan",
                       passwd=getpass.getpass("Enter MySQL passwd for jovyan"),db='mimic2')
cursor = conn.cursor()

Run the below cell. It pulls the necessary data from MIMIC-II

In [ ]:
micro = pd.read_sql("""SELECT a.subject_id
                   ,a.charttime
                   ,b.label as organism
                   ,c.label as antibiotic
                   ,a.interpretation
                   FROM microbiologyevents a
                   JOIN d_codeditems b ON a.org_itemid=b.itemid
                   JOIN d_codeditems c ON a.ab_itemid=c.itemid
                   WHERE a.interpretation != "None"
                   LIMIT 400
                   """, 
                   conn)
conn.close()

# CLASSES

Run these next two cells with the necessary classes "patient" and "susceptibility_report"

In [ ]:
class patient(object):
    """Represents a patient within the healthcare system. 
       Attributes: subject_id"""
    
    def __init__(self, subject_id):
        self.subject_id = subject_id
        
    def __str__(self):
        return "Subject ID: "+str(self.subject_id)

In [ ]:
class susceptibility_report(patient):
    """Represents a culture & sensitivity report for a patient.
       Attributes: organism, antibiotic, interpretation"""
    
    abx_interp = []
    
    def __init__(self, subject_id, organism, antibiotic, interpretation):
        super(susceptibility_report,self).__init__(subject_id)
        self.organism = organism 
        self.abx_interp = []
        self.abx_interp.append([antibiotic, interpretation])        
    
    def add_report(self, antibiotic, interpretation):
        self.abx_interp.append([antibiotic, interpretation])
    
    def get_report(self):
        return self.abx_interp
    
    def get_organism(self):
        return self.organism
    
    def get_interp(self):
        return self.interp
    
    def __str__(self):
        return super(susceptibility_report,self).__str__()+"; Organism: "+self.organism+"; Antibiotic & Susceptibility: "+str(self.abx_interp)

# BUILD SUSCEPTIBILITY REPORTS & CATEGORIZATION

Run the below cell. It builds a susceptibility report from the MIMIC-II database for a given patient and culture result. 

In [ ]:
df = pd.DataFrame(micro)
susceptibility_reports = {}

for index, row in micro.iterrows():
    if str(row['subject_id'])+" "+(row['organism'])+" "+str(row['charttime']) in susceptibility_reports.keys():
        temp = susceptibility_reports[str(row['subject_id'])+" "+(row['organism'])+" "+str(row['charttime'])]
        temp.add_report(row['antibiotic'], row['interpretation'])
    else:
        susceptibility_reports[str(row['subject_id'])+" "+(row['organism'])+" "+str(row['charttime'])]=susceptibility_report(row['subject_id'], row['organism'], row['antibiotic'], row['interpretation'])

Run the below cell. It builds a dictionary for categorizing all the antibiotics in the MIMIC-II database based on their spectrum of activity. 

In [ ]:
abx_cat = {}
abx_cat["PENICILLIN"] = "Narrow"
abx_cat["PENICILLIN G"] = "Narrow"
abx_cat["AMPICILLIN"] = "Narrow"
abx_cat["CEFAZOLIN"] = "Narrow"
abx_cat["ERYTHROMYCIN"] = "Narrow"
abx_cat["CLINDAMYCIN"] = "Narrow"
abx_cat["TRIMETHOPRIM/SULFA"] = "Narrow"
abx_cat["NITROFURANTOIN"] = "Narrow"
abx_cat["TETRACYCLINE"] = "Narrow"
abx_cat["OXACILLIN"] = "Narrow"
abx_cat["CEFUROXIME"] = "Narrow"
abx_cat["VANCOMYCIN"] = "Moderate"
abx_cat["CEFTRIAXONE"] = "Moderate"
abx_cat["AMPICILLIN/SULBACTAM"] = "Moderate"
abx_cat["RIFAMPIN"] = "Moderate"
abx_cat["CHLORAMPHENICOL"] = "Broad"
abx_cat["GENTAMICIN"] = "Broad"
abx_cat["TOBRAMYCIN"] = "Broad"
abx_cat["AMIKACIN"] = "Broad"
abx_cat["CEFTAZIDIME"] = "Broad"
abx_cat["CIPROFLOXACIN"] = "Moderate"
abx_cat["IMIPENEM"] = "Broad"
abx_cat["PIPERACILLIN"] = "Moderate"
abx_cat["LEVOFLOXACIN"] = "Moderate"
abx_cat["PIPERACILLIN/TAZO"] = "Broad"
abx_cat["CEFEPIME"] = "Broad"
abx_cat["MEROPENEM"] = "Broad"
abx_cat["DAPTOMYCIN"] = "Broad"
abx_cat["LINEZOLID"] = "Broad"

# USER INPUT REQUIRED!
Run the below cell and enter a number from the far left column when prompted. The number corresponds to an indiviudal susceptibility report. 

In [ ]:
key_list = list(susceptibility_reports.keys())
for idx, report_title in enumerate(key_list):
    print(str(idx)+" "+str(report_title))
input_num = int(input())

print("Enter a number from the far left column:")
x = key_list[input_num]

Run the below cell. It takes a susceptibility report and builds the Stewardship-Based Susceptibility Report based on both the susceptibility interpretation AND the spectrum of activity category. 

In [ ]:
preferred = []
consider = []
avoid = []
for abx,interp in susceptibility_reports[x].get_report():
    current_abx = abx+' : '+interp
    if current_abx in preferred or current_abx in consider or current_abx in avoid: 
        continue 
    if interp == 'R' or interp == 'I':
        avoid.append(current_abx)
    elif abx_cat[abx] == 'Narrow':
        preferred.append(current_abx)

timesthru = 0 
for abx,interp in susceptibility_reports[x].get_report():
    current_abx = abx+' : '+interp
    if current_abx in preferred or current_abx in consider or current_abx in avoid: 
        continue 
    if abx_cat[abx] == 'Moderate' :
        if len(preferred) > 0 and timesthru==0: 
            consider.append(current_abx)
        else:
            timesthru += 1
            preferred.append(current_abx)
            
timesthru = 0 
for abx,interp in susceptibility_reports[x].get_report():
    current_abx = abx+' : '+interp
    if current_abx in preferred or current_abx in consider or current_abx in avoid:  
        continue 
    if abx_cat[abx] == 'Broad' :
        if len(preferred) > 0 and timesthru == 0: 
            avoid.append(current_abx)
        else:
            timesthru += 1
            preferred.append(current_abx)

# STEWARDSHIP-BASED SUSCEPTIBILITY REPORT
Run the below cell to see the output of the Stewardship-Based Susceptibility Report. It categorizes the antibiotics based not only on their susceptibility (S: susceptibile, I: intermediate, or R: resistant) but also on their spectrum of activity in order to guide clinicians to select the narrowest spectrum active agent. 

In [ ]:
pref_grp = pd.DataFrame(list(zip(preferred)),columns=['Preferred'])
cons_grp = pd.DataFrame(list(zip(consider)),columns=['Consider'])
avoid_grp = pd.DataFrame(list(zip(avoid)),columns=['Avoid'])
stewie_report = pd.concat([pref_grp, cons_grp,avoid_grp], axis=1)
stewie_report.fillna(value="-", inplace=True)

print("Organism: "+susceptibility_reports[x].get_organism())
stewie_report